In [2]:
from netCDF4 import Dataset  # http://code.google.com/p/netcdf4-python/
import os
dir_in='f:/data/tc_wakes/database/info/'
dir_out='f:/data/tc_wakes/database/sst/'
#################################################################################
import datetime as dt
import xarray as xr
#from datetime import datetime
import pandas
import matplotlib as mpl
#import openpyxl
#from mpl_toolkits.basemap import Basemap
#import matplotlib.pyplot as plt
import numpy as np
import math
#from math import cos, radians
from netCDF4 import Dataset  # http://code.google.com/p/netcdf4-python/
import os
####################you will need to change some paths here!#####################
#list of input files
dir_in='f:/data/tc_wakes/database/info/'
dir_out='f:/data/tc_wakes/database/sat_data/'
dir_mur = 'F:/data/sst/jpl_mur/v4.1/'
#output files
#filename_out_nc='F:/data/cruise_data/saildrone/baja-2018/daily_files/sd-1002/data_so_far.nc'
#filename_out_kml='F:/data/cruise_data/saildrone/baja-2018/daily_files/sd-1002/data_so_far.kml'
#################################################################################
import datetime as dt
import xarray as xr
from datetime import datetime
import pandas
import matplotlib as mpl
#import openpyxl
#from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import numpy as np
import math
#from math import cos, radians
from copy import copy
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.mlab as mlab
palette = copy(plt.cm.jet)
palette.set_over('r', 1.0)
palette.set_under('g', 1.0)
palette.set_bad(alpha = 0.0)

In [3]:
#read in target lat/lon (ccmp array)
ccmp_filename='F:/data/sat_data/ccmp/v02.0/Y2003/M10/CCMP_Wind_Analysis_20031009_V02.0_L3.0_RSS.nc'
ds_target=xr.open_dataset(ccmp_filename)  #don't need nobs
ds_target_lon=ds_target.longitude
ds_target_lat=ds_target.latitude


In [4]:
dir_ccmp='F:/data/sat_data/ccmp/v02.0/Y'
date_1858 = dt.datetime(1858,11,17,0,0,0) # start date is 11/17/1958
dx=0.25
dy=0.25
dx_offset = -179.875
dy_offset = 78.3750
for root, dirs, files in os.walk(dir_in, topdown=False):
    #print(files)
    #for ii in range(40,90): 
    #print(root[31:35])
    for name in files:
        istart_storm=0
        #name = files[ii]
        fname_in=os.path.join(root, name)
        print(name,fname_in)
        dsx = xr.open_dataset(fname_in)
        lats = dsx.lat[0,:]
        lons = dsx.lon[0,:]
        dysince = dsx.time
        icycle=0
        minlon=min(lons.values)-7
        maxlon=max(lons.values)+7
        minlat=min(lats.values)-7
        maxlat=max(lats.values)+7
        if minlon<10. and maxlon>350.:  #wrapping around meridion need to cal new min/max lon
            minlon=max(lons[lons<180].values)+7
            maxlon=min(lons[lons>180].values)-7
            icycle=1 #set flag for wraparound
        dims=lats.shape
        print(dims)
        tdim=dims[0]
        tem_date=[0]*tdim #print(dysince.values)
        for i in range(0,tdim):
            tem_date[i]=date_1858+dt.timedelta(days=float(dysince[0,i].values))  #create new time array that can be queried for year etc
        minjdy = min(tem_date).timetuple().tm_yday  #create new time array that can be queried for year etc
        minyear =min(tem_date).year #create new time array that can be queried for year etc
        maxjdy = max(tem_date).timetuple().tm_yday  #create new time array that can be queried for year etc
        maxyear =max(tem_date).year  #create new time array that can be queried for year etc
        dif = max(tem_date)-min(tem_date)
        tdim=int(dif.days)  
        print(minjdy,maxjdy,minlon,maxlon,minlat,maxlat)
        ds_new=[]
        new_coord=[]
        for incr_dy in range(-5,tdim+25): #from -5 to 25 days after
            storm_date = tem_date[0]+dt.timedelta(days=incr_dy)            
            #print(storm_date)
            syr=str(storm_date.year)
            smon=str(storm_date.month)
            sdym=str(storm_date.day)
            sjdy=str(storm_date.timetuple().tm_yday)
            fname_tem=syr + smon.zfill(2) + sdym.zfill(2) + '090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc'
            mur_filename = dir_mur + syr + '/' + sjdy.zfill(3) + '/' + fname_tem
            print(mur_filename)
            ds=xr.open_dataset(mur_filename,drop_variables=('analysis_error','sea_ice_fraction'))  #don't need nobs
            ds_rolled = ds.assign_coords(lon=(ds.lon % 360)).roll(lon=(ds.dims['lon'] // 2))
            sst=ds_rolled.analysed_sst
            #interpolate 1 km mur data onto CCMP grid
            sst_interp = sst.interp(lat=ds_target_lat,lon=ds_target_lon)
            #select lat/lon region
            if icycle==0:
                ds2=sst_interp.sel(latitude=slice(minlat,maxlat),longitude=slice(minlon,maxlon))
            else:
                ds2=sst_interp.sel(latitude=slice(minlat,maxlat),longitude=(sst_interp.longitude < minlon) | (sst_interp.longitude > maxlon))
            #print(ds2)
            ds_new.append(ds2)
            new_coord.append(storm_date)
        combined = xr.concat(ds_new, dim='time')
        combined.coords['time'] = new_coord
        fname_out=dir_out + 'mur' + root[31:35] + name[0:3] + '.nc'
        print(fname_out)
        combined.to_netcdf(fname_out)
        #break
    #break
#print(ds_new)


001annual_storm_info.nc f:/data/tc_wakes/database/info/2004\001annual_storm_info.nc
(26,)
271 277 56.299998585134745 74.3999984934926 -13.999999843537807 2.8000000938773155
F:/data/sst/jpl_mur/v4.1/2003/266/20030923090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc


KeyboardInterrupt: 

In [6]:
ds

<xarray.Dataset>
Dimensions:       (lat: 17999, lon: 36000, time: 1)
Coordinates:
  * time          (time) datetime64[ns] 2003-09-23T09:00:00
  * lat           (lat) float32 -89.99 -89.98 -89.97 -89.96 -89.95 -89.94 ...
  * lon           (lon) float32 -179.99 -179.98 -179.97 -179.96 -179.95 ...
Data variables:
    analysed_sst  (time, lat, lon) float32 ...
    mask          (time, lat, lon) float32 ...
Attributes:
    Conventions:                CF-1.5
    title:                      Daily MUR SST, Final product
    summary:                    A merged, multi-sensor L4 Foundation SST anal...
    references:                 http://podaac.jpl.nasa.gov/Multi-scale_Ultra-...
    institution:                Jet Propulsion Laboratory
    history:                    created at nominal 4-day latency; replaced nr...
    comment:                    MUR = "Multi-scale Ultra-high Reolution"
    license:                    These data are available free of charge under...
    id:                    

In [2]:
import xarray as xr
fname='f:/data/model_data/gmao/flux/MERRA2_300.tavg1_2d_flx_Nx.20041114.SUB.nc4'
ds=xr.open_dataset(fname)
ds

OSError: [Errno -101] NetCDF: HDF error: b'f:\\data\\model_data\\gmao\\flux\\MERRA2_300.tavg1_2d_flx_Nx.20041114.SUB.nc4'